# Black/White income gap in cities

I was intrigued by [a FiveThirtyEight.com story](http://fivethirtyeight.com/datalab/how-baltimores-young-black-men-are-boxed-in/) that included this image:

![this image](https://espnfivethirtyeight.files.wordpress.com/2015/04/casselman-datalab-baltimore-1.png?w=610&h=521)

I wanted to know which were the places towards the bottom right quadrant, where black median income outpaced white so dramatically. I am also always looking for chances to practice Pandas and iPython notebook and see how well our [Census Reporter SQL data](http://censusreporter.tumblr.com/post/73727555158/easier-access-to-acs-data) can be applied to this kind of work, so I made this notebook.

In [15]:
# basic setup. You'd do this every time you set out to use pandas with Census Reporter's SQL
import pandas as pd
from sqlalchemy import create_engine 
# for below to work, you must set the PGPASSWORD env variable or have no-password login enabled
engine = create_engine('postgresql://census@localhost:5432/census')

After I published this, [Ben Casselman clarified](https://twitter.com/bencasselman/status/595276073077776385) that they used [B19013H Median Household Income (White alone, not Hispanic)](http://censusreporter.org/tables/B19013H/) and [B19013B Median Household Income (Black alone)](http://censusreporter.org/tables/B19013B/) tables from the ACS2013-3 year release.

In [16]:
# load in white and black median income for Census places (sumlevel = 160)
white = pd.read_sql_query("select g.geoid, g.name, d.b19013h001 as white \
                           from acs2013_3yr.geoheader g, \
                                acs2013_3yr.b19013h d \
                                where d.geoid = g.geoid \
                                and g.sumlevel = 160",engine, index_col='geoid')
black = pd.read_sql_query("select g.geoid, d.b19013b001 as black \
                           from acs2013_3yr.geoheader g, \
                                acs2013_3yr.b19013b d \
                                where d.geoid = g.geoid \
                                and g.sumlevel = 160",engine, index_col='geoid')


In [17]:
# put the parts together and compute the gap
df = white.join(black)
df = df.dropna()
df['gap'] = df.white - df.black
df.sort('gap',ascending=True,inplace=True)


The 538 chart specifies places where blacks make up 10% or more of the population. So, add population of each group. Since the median income above specifies White non-hispanic, we need to use  [B03002 Hispanic or Latino Origin by Race](http://censusreporter.org/tables/B03002/).

In [18]:
df.rename(columns={'white': 'white_income', 'black': 'black_income'}, inplace=True)
population = pd.read_sql_query("select geoid, b03002001 as total_pop, b03002003 as white_pop, \
                                b03002004+b03002014 as black_pop from acs2013_3yr.b03002 \
                                where geoid like '16000US%%'",
                               engine, index_col='geoid')
df = df.join(population)

In [19]:
df.dropna(inplace=True)
df['black_pop_pct'] = df.black_pop / df.total_pop
# I'm running out of creative names for my variables
df2 = df[(df.black_pop_pct >=.1) & (df.gap < 0)]


In [20]:
df2.head(10)

,name,white_income,black_income,gap,total_pop,white_pop,black_pop,black_pop_pct
geoid,,,,,,,,
16000US0672520,"Soledad city, California",60441,147632,-87191,26251,3391,3312,0.126167
16000US3676089,"Uniondale CDP, New York",36991,98148,-61157,25905,2416,11635,0.449141
16000US3676705,"Valley Stream village, New York",78736,122880,-44144,37618,13727,6763,0.179781
16000US3624273,"Elmont CDP, New York",63913,96223,-32310,39761,6260,19265,0.484520
16000US0611530,"Carson city, California",56728,81520,-24792,92277,6527,17385,0.188400
16000US4816468,"Converse city, Texas",54346,74891,-20545,19705,6999,3545,0.179904
16000US3627485,"Freeport village, New York",70816,85997,-15181,43095,10098,13717,0.318297
16000US0684144,"West Carson CDP, California",53826,68750,-14924,21310,4097,2438,0.114406
16000US3613552,"Central Islip CDP, New York",61287,75091,-13804,36457,7244,9076,0.248951


_The paragraphs below refer to places which are no longer on the top 10 list after I updated to align with the tables FiveThirtyEight used for their chart. They are places with a large gap in favor of the black population, but they are too small to be included in the ACS 3-year estimates. I don't have time at the moment to investigate the ones which made this top ten list._

I don't know much about Maryland, so I don't know what the deal is with [Kingstown](http://censusreporter.org/profiles/16000US2444325). It's only about [30 mi. from Baltimore](http://www.distance-cities.com/search?from=Baltimore%2C+MD&to=Kingstown%2C+MD%2C+United+States) "as the crow flies," but more than 75 by roads because of the Chesapeake Bay.

I'm from Ohio, but near Cleveland, so I never knew about [Wilberforce](http://censusreporter.org/profiles/16000US3985092-wilberforce-oh/). As the home of two historically black colleges, I am guessing that the administration and top faculty help account for the gap.



In [21]:
df2.sort('total_pop',ascending=False).head(3)

,name,white_income,black_income,gap,total_pop,white_pop,black_pop,black_pop_pct
geoid,,,,,,,,
16000US0624680,"Fontana city, California",76469,79154,-2685,201293,29249,20638,0.102527
16000US0660466,"Rialto city, California",46141,49056,-2915,101434,11722,14618,0.144113
16000US4856348,"Pearland city, Texas",86944,96335,-9391,98123,45309,17236,0.175657


It turns out that the gap for the biggest places is not too big. Maybe we should filter by some factor.

In [22]:
df3 = df2[df2.gap*-1 > df2.white_income/2]
df3.sort('total_pop',ascending=False).head() 

,name,white_income,black_income,gap,total_pop,white_pop,black_pop,black_pop_pct
geoid,,,,,,,,
16000US3624273,"Elmont CDP, New York",63913,96223,-32310,39761,6260,19265,0.484520
16000US3676705,"Valley Stream village, New York",78736,122880,-44144,37618,13727,6763,0.179781
16000US0672520,"Soledad city, California",60441,147632,-87191,26251,3391,3312,0.126167
16000US3676089,"Uniondale CDP, New York",36991,98148,-61157,25905,2416,11635,0.449141


In [23]:
# Always clean up your database
engine.dispose()